In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import os
from cleantext import clean
from pandas import DataFrame

In [112]:
import random as rnd

In [82]:
arr = os.listdir('.')
print(arr)

['Label_train_2.csv', '.DS_Store', 'Creation_matrice_Creation.ipynb', 'FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Clean_Tweets_EM.ipynb', 'Creation_matrice_Detection.ipynb', 'README.md', 'DataBase_temporary_creation.csv', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'Reseau_neurone.ipynb', 'Data_train_2.csv', 'DataElon_clean.csv', 'Vectorisation.ipynb', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


In [4]:
assert hasattr(tf, 'function')

In [5]:
Data_train = pd.read_csv("Data_train.csv")
Label_train = pd.read_csv("Label_train.csv")

In [83]:
Data_train_2 = pd.read_csv('Data_train_2.csv')
Label_train_2 = pd.read_csv("Label_train_2.csv")

In [214]:
def Construction_modele(Data_train, Label_train, affichage=1):
    #Construction couche de neurones
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))
    #Mise au bon format des donnes pour le modele
    Tweet_1 = Data_train.values
    Tweet_2 = Tweet_1.astype(dtype='float32')
    Label_1 = Label_train.values
    Label_2 = Label_1[:,0].astype(dtype='uint8')
    #Ajustement modele et choix de metrique
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    #Entrainement du modele
    model.fit(Tweet_2, Label_2, epochs=18, verbose=affichage)
    return model

In [8]:
def Vectorisation(Data_train, tweet):   #prend un tweet en argument sous la forme d'une string et renvoi son vecteur sous forme d'une database dans l espace vectoriel de la matrice
    Vecteurs = Data_train.columns
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    Mot_tweet = tweet.split(" ")
    
    for index, mot in enumerate(Mot_tweet):
            if mot in Vecteurs:
                dictionnary[mot] = index + 1
                
    row.append(dictionnary)
    Data_out = DataFrame(row)
    
    if Data_out.loc[0].max() != 0:
        Data_out.loc[0] = Data_out.loc[0] / Data_out.loc[0].max()
    
    return Data_out
    
    
    

La fonction detection prend en argument un tweet sous la forme d'un string, le reseau de neurone entraine et la base de donnee d entrainement utilise pour vectorizer le tweet. La fonction renvoi un tuple de la forme
\
\
("Elon Musk detecte a x%", 1)
\
("Indetermine", 2)
\
("Pas Elon Musk a x%", 0)
\
\
1 est renvoye ssi x >= 60%
\
0 est renvoye ssi x >= 60%
\
2 est renvoye ssi la probabilite que ce soit Elon Musk (ou non) soit comprise entre 40% et 60%

In [9]:
def detection(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        char = "Indetermine"
        return (char, 2)
    Prediction = model_neurone.predict(vecteur)
    if Prediction[0, 1] >= 0.6:
        char = "Elon Musk detecte a " + str(Prediction[0, 1]) + "%"
        return (char, 1)
    elif Prediction[0,1] >= 0.4:
        char = "Indetermine"
        return (char, 2)
    else :
        char = "Pas Elon Musk a " + str(Prediction[0, 0]) + "%"
        return (char, 0)

In [178]:
def detection_creation(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        return 0
    Prediction = model_neurone.predict(vecteur)
    return Prediction[0, 1]

In [218]:
#Detecte et rend une estimation du tweet propose et l ajoute a son entrainement
#Renvoi lestimation et le nouveau model de neurone entraine apres estimation avec le tweet donne

def detection_apprentissage(tweet_sentence, model_neurone, Data_train, Label_train, val=0):
    #estimation
    (char, val) = detection(tweet_sentence, model_neurone, Data_train)
    #re entrainement du reseau de neurones
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    Data_train = pd.concat([Data_train, vecteur_phrase], ignore_index=True)
    Data_train.reset_index(drop=True, inplace=True)
    label = DataFrame([{'label':val}])
    Label_train = pd.concat([Label_train, label], ignore_index=True)
    Label_train.reset_index(drop=True, inplace=True)
    model = Construction_modele(Data_train, Label_train, affichage=0)
    return ((char, val), model)

In [223]:
(3,4)[1]

4

In [ ]:
def detection_apprentissage_liste(tweet_liste, model_neurone, Data_train, Label_train, val_list):
    Estimat = []
    for tweet in tweet_liste:
        Estimat.append(detection(tweet, model_neurone, Data_train)[1])
        

In [81]:
#Initialisation des reseaux de neurones avant le dialogue

In [217]:
reseau_neurone_detection = Construction_modele(Data_train, Label_train)

Epoch 1/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6897 - accuracy: 0.5312
Epoch 2/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6727 - accuracy: 0.6542
Epoch 3/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6461 - accuracy: 0.7289
Epoch 4/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6007 - accuracy: 0.7595
Epoch 5/18
348/348 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7827
Epoch 6/18
348/348 [==============================] - 1s 1ms/step - loss: 0.4769 - accuracy: 0.7995
Epoch 7/18
348/348 [==============================] - 1s 1ms/step - loss: 0.4270 - accuracy: 0.8188
Epoch 8/18
348/348 [==============================] - 1s 2ms/step - loss: 0.3905 - accuracy: 0.8359
Epoch 9/18
348/348 [==============================] - 1s 2ms/step - loss: 0.3628 - accuracy: 0.8445
Epoch 10/18
348/348 [==============================] - 1s 2ms/step - loss: 0.3414 - accuracy: 0.8562

In [84]:
reseau_neurone_creation = Construction_modele(Data_train_2, Label_train_2)

Epoch 1/18
370/370 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5521
Epoch 2/18
370/370 [==============================] - 0s 1ms/step - loss: 0.6754 - accuracy: 0.6146
Epoch 3/18
370/370 [==============================] - 0s 1ms/step - loss: 0.6579 - accuracy: 0.6694
Epoch 4/18
370/370 [==============================] - 0s 1ms/step - loss: 0.6314 - accuracy: 0.7017
Epoch 5/18
370/370 [==============================] - 0s 1ms/step - loss: 0.5976 - accuracy: 0.7197
Epoch 6/18
370/370 [==============================] - 0s 1ms/step - loss: 0.5625 - accuracy: 0.7385
Epoch 7/18
370/370 [==============================] - 0s 1ms/step - loss: 0.5305 - accuracy: 0.7559
Epoch 8/18
370/370 [==============================] - 0s 1ms/step - loss: 0.5031 - accuracy: 0.7691
Epoch 9/18
370/370 [==============================] - 0s 1ms/step - loss: 0.4792 - accuracy: 0.7792
Epoch 10/18
370/370 [==============================] - 0s 1ms/step - loss: 0.4581 - accuracy: 0.7935

In [89]:
#Debut du dialogue

In [209]:
def Creation_tweet(nombre_mots_total, phrase, Data_train_2, model_neurone): #renvoi un tweet sous la forme d une phrase 
    phrase = phrase.lower()
    phrase = clean(phrase, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    liste_mot =  phrase.split(" ")
    if nombre_mots_total < len(liste_mot):
        print("Erreur liste mot plus longue que le tweet prevu, longueur du tweet pris par defaut au nombre de mots")
        nombre_mots_total = len(liste_mot)
    nombre_mot_rest = nombre_mots_total - len(liste_mot)
    
    liste_tweet = liste_mot + rnd.sample(list(Data_train_2.columns), nombre_mot_rest, counts=None)
    tweet_pertinent = ' '.join(liste_tweet)
    pertinence = detection_creation(tweet_pertinent, model_neurone, Data_train_2)
    
    for iteration in range(100):
        if pertinence > 0.9:
            return tweet_pertinent
        liste = liste_mot + rnd.sample(list(Data_train_2.columns), nombre_mot_rest, counts=None)
        rnd.shuffle(liste)
        current_tweet = ' '.join(liste)
        current_pertinence = detection_creation(current_tweet, model_neurone, Data_train_2)
        if current_pertinence > pertinence:
            pertinence = current_pertinence
            tweet_pertinent = current_tweet
    return current_tweet
        
    

In [222]:
tweetos = Creation_tweet(8, 'i love madonna', Data_train_2, reseau_neurone_creation)
tweetos

'love i mr madonna actually artpop giant update'

In [211]:
detection(tweetos, reseau_neurone_creation, Data_train_2)

('Elon Musk detecte a 0.92738056%', 1)

In [ ]:
def dialogue(iteration, nombre_mot ,reseau_neurone_detection, reseau_neurone_creation, Data_train, Data_train_2, Label_train, Label_train_2):
    for i in range(iteration)

In [219]:
phrase = 'I sing every morning in my shower, i love '
#reponse = detection(phrase, reseau_neurone_creation, Data_train_2)
reponse = detection_apprentissage(phrase, reseau_neurone_detection, Data_train, Label_train)


In [213]:
((_, val), _) = reponse
val

0